In [29]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import seaborn as sns

In [2]:
df = pd.read_csv('test_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   transaction_date  100000 non-null  object 
 1   property_id       100000 non-null  int64  
 2   property_type     100000 non-null  object 
 3   region            100000 non-null  object 
 4   price             100000 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 3.8+ MB


In [4]:
df['transaction_date'] =  pd.to_datetime(df['transaction_date'], format='%Y-%m-%d')

In [5]:
df.head()

,transaction_date,property_id,property_type,region,price
0,2020-01-01,512624,Condo,Metro Vancouver,812353.75
1,2020-01-01,499817,Condo,Nanaimo,261333.66
2,2020-01-01,388958,Townhouse,Central Okanagan,744377.10
3,2020-01-01,708544,Townhorse,Nanaimo,555737.79
4,2020-01-01,227012,Single Family Dwelling,Metro Vancouver,1186354.04


In [6]:
dfs = df.groupby(['property_id']).size().reset_index(name='cnt')

In [7]:
dfs = dfs[dfs['cnt'] > 1]

In [8]:
dfs.reset_index(inplace=True, drop=True)
dfs[:200]

,property_id,cnt
0,100063,2
1,100113,2
2,100260,5
3,100266,3
4,100306,2
...,...,...
195,107735,2
196,107965,2
197,108019,2
198,108022,4


In [22]:
# result_template = dict(transaction_date=[], property_id=[], property_type=[], region=[], price=[])
cols = ['property_id', 'property_type', 'region', 'transction_date_1', 'price_1', 'transaction_date_2', 'price_2', 'date_diff']
pairs_30 = pd.DataFrame(columns=cols)
pairs_90 = pairs_30.copy()
pairs_180 = pairs_30.copy()
for index, row in tqdm(dfs.iterrows(), total=dfs.shape[0]):
    pid = row['property_id']
    subset = df[df['property_id'] == pid]
    subset.reset_index(inplace=True, drop=True)
    for i, r in subset.iterrows():
        for j in range(i + 1, len(subset)):
            date_diff = subset.iloc[j, 0] - r['transaction_date']
            date_diff = date_diff.days
            lst = [r['property_id'], r['property_type'], r['region'], r['transaction_date'], r['price'], subset.iloc[j, 0], subset.iloc[j, 4], date_diff]
            log_row = pd.Series(lst, index=cols)
            if (date_diff <= 30):
                pairs_30 = pairs_30.append(log_row, ignore_index=True)
            if (date_diff <= 90):
                pairs_90 = pairs_90.append(log_row, ignore_index=True)
            if (date_diff <= 180):
                pairs_180 = pairs_180.append(log_row, ignore_index=True)

100%|██████████| 8325/8325 [00:23<00:00, 359.34it/s]


In [26]:
pairs_30.to_csv('pairs_30.csv')
pairs_90.to_csv('pairs_90.csv')
pairs_180.to_csv('pairs_180.csv')

In [74]:
# dfs = df[df.groupby(['property_id'])['region'].transform('size') > 1]

In [49]:
# df1 = df[df['property_id'].duplicated(keep=False) == True]

In [27]:
pairs_180.corr(method='pearson')

,price_1,price_2
price_1,1.000000,0.999645
price_2,0.999645,1.000000
